I tried to just load the mhd files directly before doing any preprocessing, but then the kernel died after 21 images loaded, so I will be doing preprocessing directly after loading the image and then saving that smaller array. When printing the shape of the ct scans I could also see that each slize was 512*512 but that a different number of slices was used for each ct scan, from 100 to 500, so it could differ quite a lot. I don´t know if the slice number has a certain significance. Looking at this article: https://soundvet.com/portable-ct/understanding-ct-scanner-specifications-slice-count-scan-speed-and-cost-efficiency/ it seems like a whole ct scanner might be set to a certain number of slices, but maybe there are scanners where you can adjust the number of slices? According to this article: https://lbnmedical.com/ct-scan-slice-types/ a CT scanner takes 4 - 640 slices. According to the article it seems like our ct scans with over 100 slices are very detailed, have very high image quality and have been taken with expensive machines - alright. I´m still a bit confused about the irregular number of slices, I would expect powers of 2, maybe 128,256 etc but I´m seeing 204, 157, 139 etc - maybe I´ll figure that one out later. 

In [ ]:
#SimpleITK helps us load the mhd files into numpy arrays

import SimpleITK as sitk
import numpy as np
import glob
import os

local_path = '/home/ec2-user/SageMaker/images/'

def load_mhd_file(file_path):
    # Read the MHD file using SimpleITK
    image = sitk.ReadImage(file_path)
    
    # Convert it to a NumPy array (H x W x D)
    image_array = sitk.GetArrayFromImage(image)
    
    print(image_array.shape)
    
    return image_array

image_arrays = []
for filename in glob.glob(os.path.join(local_path, '*.mhd')):
    image_data = load_mhd_file(filename)
    image_arrays.append(image_data)

print(image_arrays)

# Check the shape of the loaded image (should be 3D)
#print(image_arrays[0].shape)

(204, 512, 512)
(157, 512, 512)
(139, 512, 512)
(133, 512, 512)
(246, 512, 512)
(265, 512, 512)
(133, 512, 512)
(157, 512, 512)
(139, 512, 512)
(245, 512, 512)
(133, 512, 512)
(133, 512, 512)
(588, 512, 512)
(119, 512, 512)
(123, 512, 512)
(183, 512, 512)
(232, 512, 512)
(672, 512, 512)
(121, 512, 512)
(151, 512, 512)
(280, 512, 512)
(483, 512, 512)
(280, 512, 512)
(127, 512, 512)
(140, 512, 512)
(129, 512, 512)
(124, 512, 512)
